In [2]:
import tkinter
import numpy as np
import torch

In [9]:
# Slice origin point
# center_point = np.matrix([5,5]).T
center_point = torch.tensor([[5],[5]])

# Slice angles
# theta1 = 3* np.pi / 2
# theta2 = 3.9* np.pi / 2
# theta3 = np.pi / 1.3
theta1 = torch.tensor(3 * np.pi / 2, requires_grad=True)
theta2 = torch.tensor(3.9 * np.pi / 2, requires_grad=True)
theta3 = torch.tensor(np.pi / 1.3, requires_grad=True)

# origin = np.matrix([0,0]).T
origin = torch.tensor([[0],[0]])

# Slices must be longer than 500^.5. Slices creater by rotating starter
# slice_starter = np.matrix([30, 0]).T

# # Creates a rotation matrix from a given theta
# rotate_mat = lambda theta : np.matrix([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]]) 
# # Creates slice vector
# get_slice = lambda theta : rotate_mat(theta) * slice_starter + center_point

# # Create the slices
# slice1 = get_slice(theta1)
# slice2 = get_slice(theta2)
# slice3 = get_slice(theta3)

# # Corners of the cake
# upper_left  = np.matrix([0,10]).T
# upper_right = np.matrix([20,10]).T
# lower_left  = np.matrix([0,0]).T
# lower_right = np.matrix([20,0]).T

In [3]:
def find_slice_intersection(slice, center_point, vec2, shift2):
    vec1 = (slice - center_point).T
    vec2 = vec2.T
    shift1 = center_point.T
    shift2 = shift2.T

    t = np.cross((shift2 - shift1), vec2) / np.cross(vec1, vec2)
    u = np.cross((shift2 - shift1), vec1) / np.cross(vec1, vec2)

    if np.abs(np.cross(vec1, vec2)) < 1e-10:
        # The lines are parallel
        return False
    elif t > 0 and t < 1 and u > 0 and u < 1:
        # The lines intersect
        return (t*vec1 + shift1).T
    else:
        # The lines do not intersect
        return False
    
def get_intersection_point(slice, center_point):
    intersection_top = find_slice_intersection(slice, center_point, lower_right, upper_left)
    intersection_left = find_slice_intersection(slice, center_point, upper_left, origin)
    intersection_right = find_slice_intersection(slice, center_point, upper_left, lower_right)
    intersection_bottom = find_slice_intersection(slice, center_point, lower_right, origin)

    # Hopefully this doen't break
    if intersection_top is not False:
        return intersection_top
    elif intersection_left is not False:
        return intersection_left
    elif intersection_right is not False:
        return intersection_right
    elif intersection_bottom is not False: 
        return intersection_bottom
    else:
        return False

In [20]:
slice1_intersection = get_intersection_point(slice1, center_point)
slice2_intersection = get_intersection_point(slice2, center_point)
slice3_intersection = get_intersection_point(slice3, center_point)

def get_slice_perimeters(slice1_intersection, slice2_intersection, slice3_intersection):
    perims = np.zeros(3)

    slice1_trav_dist = get_traversal_distance(slice1_intersection)
    slice2_trav_dist = get_traversal_distance(slice2_intersection)
    slice3_trav_dist = get_traversal_distance(slice3_intersection)

    traversal_dists = np.sort([slice1_trav_dist, slice2_trav_dist, slice3_trav_dist])

    print(traversal_dists)
    perims = np.array([traversal_dists[1] - traversal_dists[0],
                       traversal_dists[2] - traversal_dists[1],
                       60 - traversal_dists[2] + traversal_dists[0]])
    
    return perims


# Returns the amount of distance that you would need to travel when starting at (0,0) and 
# traversing clockwise around the cake until you reach the intersection point
def get_traversal_distance(slice_intersection):
    if slice_intersection[0,0] == 0:
        # Intersection is on the left side of the rectangle
        return slice_intersection[1,0]
    elif slice_intersection[1,0] == 10:
        # Intersection is on the top of the rectangle
        return 10 + slice_intersection[0,0]
    elif slice_intersection[0,0] == 20:
        # Intersection is on the right side of the rectangle
        return 30 + (10 - slice_intersection[1,0])
    else:
        # Intersection is on the bottom of the rectangle
        return 40 + (20 - slice_intersection[0,0])

[ 9.42961347 37.3757666  55.        ]


array([27.94615314, 17.6242334 , 14.42961347])

In [14]:
# Scaling and shifting is done for visualization purposes. tkiniter has an inverse y axis
scale_factor = np.matrix([[1,0],[0,-1]])
scale_factor *= 5
shift = np.matrix([100, 100]).T

# Transforms points to graphics space
graphics_transform = lambda point : scale_factor * point + shift

# Transform all points and draw lines
center_point_g = graphics_transform(center_point)

slice1_g = graphics_transform(slice1)
slice2_g = graphics_transform(slice2)
slice3_g = graphics_transform(slice3)

upper_left_g = graphics_transform(upper_left)
upper_right_g = graphics_transform(upper_right)
lower_left_g = graphics_transform(lower_left)
lower_right_g = graphics_transform(lower_right)

window = tkinter.Tk()
canva = tkinter.Canvas(window)

line = canva.create_line(*center_point_g.T.tolist()[0], *slice1_g.T.tolist()[0])
line = canva.create_line(*center_point_g.T.tolist()[0], *slice2_g.T.tolist()[0])
line = canva.create_line(*center_point_g.T.tolist()[0], *slice3_g.T.tolist()[0])

line = canva.create_line(*upper_left_g.T.tolist()[0], *upper_right_g.T.tolist()[0])
line = canva.create_line(*upper_left_g.T.tolist()[0], *lower_left_g.T.tolist()[0])
line = canva.create_line(*lower_left_g.T.tolist()[0], *lower_right_g.T.tolist()[0])
line = canva.create_line(*lower_right_g.T.tolist()[0], *upper_right_g.T.tolist()[0])

canva.pack()
window.mainloop()